# Weekly Momentun Scanner Executed Weekly

In [2]:
import yfinance as yf
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles.numbers import FORMAT_PERCENTAGE_00
from openpyxl.utils import get_column_letter
from stock_config import nifty50_stock_list, nifty500_stock_list, momentum_stock_list, nse_fno_stock_list, sample_stock_list

#### Function to check valid trading day. If open price is not returned then PREVIOUS date is checked until valid date is found.
##### End date is valid date + 1. Used to get LTP for desired date using yfinance.

In [3]:
def get_valid_date(date):
    valid = False
    while not valid:
        end = (pd.to_datetime(date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
        valid_check_data = yf.download('RELIANCE.NS', start = date, end=end)
        try:
            open_price = valid_check_data.iloc[0]['Open']
            valid = True
        except:
            date = (pd.to_datetime(date) - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
            valid = False
    return date

#### Function to check valid trading day with *LOCAL* data. If open price is not returned then PREVIOUS date is checked until valid date is found.

In [4]:
def get_valid_date_local(df, date):
    valid = False
    while not valid:
        end = (pd.to_datetime(date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
        try:
            open_price = df.loc[date]['Adj Close']
            valid = True
        except:
            date = (pd.to_datetime(date) - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
            valid = False
    return date

#### Get Stock Data using Yahoo Finance API for the required period

In [5]:
def get_stock_data(stock_list, start_date, end_date):
    print(f"Total stocks in the list {len(stock_list)}")
    
    #end date should be always 1 day ahead
    end_date = (pd.to_datetime(end_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
    
    data = yf.download(stock_list, start = start_date, end=end_date)
    return data

### Run test for a given evaluation date

In [6]:
def run_test(stock_data, eval_date):
    prev_week_date = (pd.to_datetime(eval_date) - pd.DateOffset(days=7)).strftime('%Y-%m-%d')
    prev_week_date = get_valid_date_local(stock_data, date=prev_week_date)
    
    this_week_date = eval_date
    this_week_date = get_valid_date_local(stock_data, date=this_week_date)
    
    next_week_date = (pd.to_datetime(eval_date) + pd.DateOffset(days=7)).strftime('%Y-%m-%d')
    next_week_date = get_valid_date_local(stock_data, date=next_week_date)  
    
    prev_week_close = pd.DataFrame(stock_data.loc[prev_week_date]['Adj Close'])
    this_week_close = pd.DataFrame(stock_data.loc[this_week_date]['Adj Close'])
    next_week_close = pd.DataFrame(stock_data.loc[next_week_date]['Adj Close'])
    
    # Create a dataframe from the series' data obtained above
    df = prev_week_close
    df = df.join(this_week_close)
    df = df.join(next_week_close)
    df.rename(columns =lambda t: t.strftime('%Y-%m-%d'), inplace=True)

    # Calculate Momentum
#     prev_week_date = df.columns[0]
#     this_week_date = df.columns[1]
#     next_week_date = df.columns[2]
    df['Change(C-B)'] = df[this_week_date] - df[prev_week_date]
    df['% Change'] = (df['Change(C-B)'] / df[prev_week_date])*100
    df['Gain(D-B)'] = df[next_week_date] - df[this_week_date]
    df['% Gain'] = (df['Gain(D-B)'] / df[this_week_date])*100
    
    # Final Output
    df.sort_values(by=['% Change'], ascending=False, inplace=True)
    df.rename_axis('MOMENTUM_ATH', inplace=True)
    df.rename_axis('', axis='columns', inplace=True)   
    
    return df

In [18]:
def write_to_excel(df, filename):
    book = load_workbook(filename)
    writer = pd.ExcelWriter(filename, engine = 'openpyxl')
    writer.book = book
    
    sheet_name = df.columns[1]
    df.round(2).to_excel(writer, sheet_name=sheet_name)
    writer.save()
    writer.close()
    print(f"{sheet_name} Done. Output file path and name --> {filename}") 
    
    # Insert formulas to calculate gains of to 20 stocks
    sheet = book.get_sheet_by_name(sheet_name) 
    sheet['J20'] = '=SUM(C2:C20)'  
    sheet['K20'] = '=SUM(G2:G20)'
    sheet['M20'] = '=K20/J20'
    sheet['M20'].number_format = FORMAT_PERCENTAGE_00

    for column_cells in sheet.columns:
        sheet.column_dimensions[get_column_letter(column_cells[0].column)].width = 12    
    sheet.column_dimensions['A'].width = 20

    book.save(filename)  
    

# Main Function

### Input the date range (start every friday date), stock universe and output filename

In [32]:
stock_list = momentum_stock_list
filename = r'data/output/momentum_weekly_output.xlsx'
start_date = '2020-12-04'
end_date = '2020-12-11'

#### Get data range. Data range starts from 1 week before start date for these calculations

In [33]:
data_start_date = (pd.to_datetime(start_date) - pd.DateOffset(days=7)).strftime('%Y-%m-%d')

#### Get Stock Data for given list of stocks

In [34]:
stock_data = get_stock_data(stock_list, start_date = data_start_date, end_date=end_date)

Total stocks in the list 627
[*********************100%***********************]  627 of 627 completed


#### Run test for the collected data

In [23]:
final_df = run_test(stock_data, start_date)

#### Output data to excel file (Create an empty excel file as below filename before running below code)

In [ ]:
write_to_excel(final_df, filename)

## Use below block only to run in a loop

In [35]:
while (start_date < end_date):
    final_df = run_test(stock_data, start_date)
    write_to_excel(final_df, filename)
    start_date = (pd.to_datetime(start_date) + pd.DateOffset(days=7)).strftime('%Y-%m-%d')

2020-12-04 Done. Output file path and name --> data/output/momentum_weekly_output.xlsx


ValueError: columns overlap but no suffix specified: DatetimeIndex(['2020-12-11'], dtype='datetime64[ns]', freq=None)

In [13]:
end_date

'2020-01-24'